In [1]:
## import all necessary packages
import json
import re
import pandas as pd

import nltk, string
from nltk.collocations import *

import numpy as np
import matplotlib.pyplot as plt

from pandas import *

from collections import defaultdict

import data_cleaning_hr as tc

import string
import happyfuntokenizing
import json

In [2]:
with open("./data/categories_manual.json") as json_file:
    category_dict = json.load(json_file)

In [3]:
with open('./data/emoji_webscraped_expanded.json','r') as f_emoji:
    emoji_df = DataFrame(json.load(f_emoji))

In [4]:
with open('./data/tweets_training_clean.json','r') as f:
    tweet_df = DataFrame(json.load(f))

In [5]:
tweet_df.head()

,id,only_HashTag,only_emoji,only_text_splithashtag,retweet,split_hashtag,text,user_id
0,383560784879833088,[],[♥],"I'm at SEPTEMBER ( Sunnyvale , CA ) url",None,[],"I'm at SEPTEMBER♥ (Sunnyvale, CA) url",47880944
1,385148713460453376,[],[],oops . fell asleep in the library . class started at 2 .,None,[],oops. fell asleep in the library. class started at 2.,259008472
10,380511446926635008,[],[],Time for a splurge with the co-workers . ( @ DishDash Restaurant w / 2 others ) url,None,[],Time for a splurge with the co-workers. (@ DishDash Restaurant w/ 2 others) url,777228427
100,379601897100812289,[],[],I slept hella late anyways .,None,[],I slept hella late anyways.,1104686922
1000,378303709446295552,[],"[😎, 🙏]",walking with my headphones on kinda high ~ ok ~ I'm feeling today,None,[],walking with my headphones on kinda high ~ ok ~ I'm feeling today 😎🙏,597925598


## Statistics on all emojis

**emoji_array**:  list of all emojis

In [6]:
emoji_array = [word for item in tweet_df.only_emoji for word in item]
len(emoji_array)

298310

**unique_emoji**:  number of unique emojis

In [7]:
unique_emoji = set(emoji_array)
len(unique_emoji)

771

**full_dict**: percentage of emojis by emoji

In [8]:
full_dict = defaultdict(int)
for item in emoji_array:
    full_dict[item] += 1

In [9]:
for item in sorted(full_dict.items(), key=lambda x:x[1], reverse=True)[:50]:
    print(item[0], (float(item[1])/len(emoji_array))*100)

😂 12.420971472629144
😍 5.247561261774663
😭 4.42962019375817
😃 4.2992189333243935
❤ 3.1963393784988767
😘 2.4524823170527306
👌 2.1739130434782608
😞 1.906406087626965
😏 1.7837149274244912
😒 1.7642720659716404
😊 1.7585732962354599
💕 1.658677214977708
😩 1.6395695752740438
😁 1.6043712916093997
😉 1.4545271697227717
👏 1.4300559820321141
😔 1.4189936643089403
☺ 1.4169823338138179
🙌 1.3492675404780263
😳 1.282893634138983
🎶 0.9151553752807483
🙏 0.8709061043880526
😕 0.8306794944856023
👍 0.8236398377526735
😋 0.8152592940229961
💁 0.7260903087392311
😑 0.7244141999932956
🎉 0.7086587777815024
✌ 0.7059770037880058
😌 0.6818410378465355
😎 0.6610572893969361
😡 0.645301867185143
😴 0.564848647380242
🙈 0.5584794341456873
😱 0.5420535684355201
💛 0.5403774596895846
😫 0.5400422379403975
😀 0.5370252421977137
💙 0.5209345982367336
🔫 0.5105427240119339
😛 0.5038382890281922
♥ 0.49277597130501827
😄 0.4910998625590828
😷 0.47165700110623177
😜 0.46126512688143206
😢 0.4579129093895612
💋 0.4267372867151621
💜 0.40461265126881

## Statistics on only faces

**unique_face**: unique faces from category_dict file

In [10]:
unique_face = [face for item in category_dict.values() for face in item]
len(unique_face)

80

**all_faces**: from emoji_array (list of all emojis), subset by unique_face

In [11]:
all_faces = [unique for face in emoji_array for unique in unique_face if face in unique]
len(all_faces)

182803

In [12]:
all_faces_unique = set(all_faces)
len(all_faces_unique)

80

**categorize**: list of all_faces defined by their respective category from cateogry_dict

In [13]:
cateogrize = []
for category, faces in category_dict.items():
    for every_face in all_faces:
            if every_face in faces:
                cateogrize.append((category, every_face))

In [14]:
# cateogrize

**categorize_percentage**: dictionary with count of each categroy form categorize

In [15]:
category_percentage = defaultdict(int)
for item in cateogrize:
    category_percentage[item[0]] += 1

In [16]:
for item in sorted(category_percentage.items(), key=lambda x:x[1], reverse=True)[:50]:
    print(item[0], (float(item[1])/len(cateogrize))*100)

happy 48.710360333254926
despairing 35.71385589952025
love 13.194531818405606
nonhuman 2.3812519488192208


## Add category column to dataframe

In [59]:
def cateogry_column(text):
    if len(text) == 0:
        return None
    
    boolean_face = [True if face in unique_face else False for face in text]
#     print(boolean_face)
    
    for item in boolean_face:
        if all(boolean_face) == True:
        
            cateogrize = []
            for category, faces in category_dict.items():
                for face in text:
                        if face in faces:
                            cateogrize.append(category)
            cateogrize = list(set(cateogrize))
            if len(cateogrize)==1:
                return cateogrize[0]
            else:
                return "Mix_P"
                
            return list(set(cateogrize))
        
        elif all(not element for element in boolean_face):
            return "Other"
#         else:
#             return "Mix_P_NonP"
        else:
            true_boolean = [index for index, item in enumerate(boolean_face) if item == True]
            keep_true = [text[i] for i in true_boolean]
            
            cateogrize = []
            for category, faces in category_dict.items():
                for face in keep_true:
                        if face in faces:
                            cateogrize.append(category)
            cateogrize = list(set(cateogrize))
            if len(cateogrize)==1:
                return cateogrize[0]
            else:
                return "Mix_P"

    return

In [63]:
print(cateogry_column([]))
print(cateogry_column(["❤"]))
print(cateogry_column(["👌"]))
print(cateogry_column(["😒", "😀"]))
print(cateogry_column(["😒", "👌"]))
print(cateogry_column(["😒","❤","😩", "👌"]))

None
Other
Other
Mix_P
despairing
despairing


In [64]:
tweet_df["cateogry_prediction"] = tweet_df.only_emoji.apply(cateogry_column)

In [65]:
dict_category_prediction = defaultdict(int)
for item in tweet_df.cateogry_prediction:
    dict_category_prediction[item] += 1

In [66]:
dict_category_prediction

defaultdict(int,
            {'love': 11559,
             'Mix_P': 8694,
             'Other': 41217,
             'happy': 52558,
             None: 625122,
             'despairing': 38702,
             'nonhuman': 2345})

In [67]:
(11559 + 52558 + 38702 + 2345) / (len(tweet_df) - 625122)

0.678149282605191

In [68]:
(11559 + 52558 + 38702 + 2345)

105164

## Make Final column numeric

In [69]:
tweet_df.head()

,id,only_HashTag,only_emoji,only_text_splithashtag,retweet,split_hashtag,text,user_id,cateogry_prediction,category_numeric
0,383560784879833088,[],[♥],"I'm at SEPTEMBER ( Sunnyvale , CA ) url",None,[],"I'm at SEPTEMBER♥ (Sunnyvale, CA) url",47880944,Other,6
1,385148713460453376,[],[],oops . fell asleep in the library . class started at 2 .,None,[],oops. fell asleep in the library. class started at 2.,259008472,None,7
10,380511446926635008,[],[],Time for a splurge with the co-workers . ( @ DishDash Restaurant w / 2 others ) url,None,[],Time for a splurge with the co-workers. (@ DishDash Restaurant w/ 2 others) url,777228427,None,7
100,379601897100812289,[],[],I slept hella late anyways .,None,[],I slept hella late anyways.,1104686922,None,7
1000,378303709446295552,[],"[😎, 🙏]",walking with my headphones on kinda high ~ ok ~ I'm feeling today,None,[],walking with my headphones on kinda high ~ ok ~ I'm feeling today 😎🙏,597925598,happy,0


In [70]:
numeric_category = [(index, value) for index, value in enumerate(list(set(tweet_df.cateogry_prediction)))]
numeric_category

[(0, 'Mix_P'),
 (1, 'despairing'),
 (2, 'nonhuman'),
 (3, 'love'),
 (4, 'happy'),
 (5, 'Other'),
 (6, None)]

In [71]:
def make_numeric(text):
    _num = [item[0] for item in numeric_category if text == item[1]]
    return _num[0]

In [76]:
make_numeric(None)

6

In [77]:
tweet_df['category_numeric'] = tweet_df.cateogry_prediction.apply(make_numeric)

In [79]:
tweet_df.shape

(780197, 10)

In [80]:
tweet_df.to_json('./data/tweets_training_clean_preprocessing.json', force_ascii=False)